<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Three_Level_Maser_Dynamics_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt

# Parameters setup for three-level maser
omega_i_values = [1.50e11, 1.55e11, 1.60e11, 1.65e11]
tau_values = [1e-9, 2e-9, 3e-9, 4e-9]

# Initial state for a three-level maser (example state; can be modified)
psi = np.array([1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)])
rho_0 = np.outer(psi, np.conj(psi))

# Hamiltonian for three-level maser (example; can be modified)
H_i = [omega * np.array([[0, 0, 0], [0, 1, 0], [0, 0, 2]]) for omega in omega_i_values]

# Validation functions
def is_hermitian(matrix, tol=1e-10):
    return np.allclose(matrix, matrix.conj().T, atol=tol)

def has_trace_1(matrix, tol=1e-10):
    return np.isclose(np.trace(matrix), 1, atol=tol)

def is_positive_semidefinite(matrix, tol=1e-10):
    return np.all(np.linalg.eigvals(matrix) >= -tol)

# Validate initial density matrix
assert is_hermitian(rho_0), "rho_0 is not Hermitian"
assert has_trace_1(rho_0), "rho_0 does not have trace 1"
assert is_positive_semidefinite(rho_0), "rho_0 is not positive semi-definite"

# Validate Hamiltonians
for H in H_i:
    assert is_hermitian(H), "A Hamiltonian is not Hermitian"

# Functions to calculate different quantities
def unitary(H, tau):
    """Calculate the unitary operator given Hamiltonian and tau."""
    return la.expm(-1j * H * tau)

def rho_after_unitary(U, rho):
    """Calculate the density matrix after applying unitary operation."""
    return U @ rho @ np.conj(U).T

def entropy(rho):
    """Calculate von Neumann entropy."""
    eigenvalues = la.eigvalsh(rho)
    return -np.sum(eigenvalues * np.log2(eigenvalues + 1e-10))

def work(rho_i, rho_f, H):
    """Calculate work done."""
    return np.trace(rho_f @ H) - np.trace(rho_i @ H)

# Simulate cycle
rho = [rho_0]
entropies = [entropy(rho_0)]
works = []

for i in range(4):
    U = unitary(H_i[i], tau_values[i])
    rho.append(rho_after_unitary(U, rho[-1]))
    entropies.append(entropy(rho[-1]))
    works.append(work(rho[i], rho[i+1], H_i[i]))

# Calculate total work and thrust
W_total = np.sum(works)
T_cycle = np.sum(tau_values)
thrust = W_total / T_cycle

# Visualization
plt.figure(figsize=(10,6))
plt.plot(range(len(entropies)), entropies, marker='o', label='Entropy')
plt.plot(range(1, len(works) + 1), works, marker='x', linestyle='--', label='Work per step')
plt.axhline(y=W_total, color='r', linestyle='--', label='Total work')
plt.xlabel('Step')
plt.ylabel('Value')
plt.title(f'Total work done: {W_total:.2f}, Thrust: {thrust:.2f}')
plt.legend()
plt.grid(True)
plt.show()
